# Gold taulujen rakentelua

In [1]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
import os
import json
import numpy as np
from dotenv import load_dotenv
from azure.cosmos import CosmosClient, PartitionKey
import polars as pl 
import pandas as pd


load_dotenv('../../shared/.env')

# database env
URL = os.getenv('COSMOSDB_ENDPOINT')  
MASTER_KEY = os.getenv('COSMOSDB_KEY')

# init client
client = cosmos_client.CosmosClient(URL, {'masterKey': MASTER_KEY}, user_agent="", user_agent_overwrite=True)

# Create a database (if it doesn't exist)
database_name = 'MojovaDB'
database = client.get_database_client(database_name)

# Create a container (if it doesn't exist)
container_name = 'Analytics'
try:
    container = database.create_container_if_not_exists(
        id=container_name,
        partition_key=PartitionKey(path='/medallion'),
        offer_throughput=400
    )
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)


In [2]:
def query(q):
    data = container.query_items(query=q, enable_cross_partition_query = True)
    data = [item for item in data]
    return pd.DataFrame(data)



In [3]:
## Kooste HOPP query ##
q = """
SELECT 
    * 
FROM 
    c 
WHERE 
    c['/medallion'] = 'silver_hopp' AND c.vastauksia != null
"""
df_kooste_hopp = query(q)
df_kooste_hopp

,org_code,unit_code,quarter,1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin,2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa,3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta,4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe,5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille,6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan,7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista,...,6_hoitajat_kertoivat_mie_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan,7_hoitajat_kertoivat_mie_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista,11_hoitajat_huolehtivat_etteivat_hoito_ja_tai_tutkimukset_aiheuttaneet_mie_noloja_tai_kiusallisia_tilanteita,unnamed:_25,/medallion,_rid,_self,_etag,_attachments,_ts
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HrfAQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""21004b80-0000-4700-0000-673e4c670000""",attachments/,1732136039
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HrgAQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""21004c80-0000-4700-0000-673e4c670000""",attachments/,1732136039
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HrhAQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""21004e80-0000-4700-0000-673e4c670000""",attachments/,1732136039
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HriAQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""21005280-0000-4700-0000-673e4c670000""",attachments/,1732136039
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HrjAQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""21005380-0000-4700-0000-673e4c670000""",attachments/,1732136039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17070,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HoYTQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""020115c1-0000-4700-0000-67518db70000""",attachments/,1733397943
17071,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HoZTQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""020119c1-0000-4700-0000-67518db70000""",attachments/,1733397943
17072,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HoaTQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""02011cc1-0000-4700-0000-67518db70000""",attachments/,1733397943
17073,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,silver_hopp,1kAqALc91HobTQAAAAAAAA==,dbs/1kAqAA==/colls/1kAqALc91Ho=/docs/1kAqALc91...,"""02011ec1-0000-4700-0000-67518db70000""",attachments/,1733397943


In [4]:
## HOPP kyssarit query
q = """
SELECT 
    * 
FROM 
    c 
WHERE 
    c['/medallion'] = 'silver_hopp' AND c.vastauksia = null
"""
df_kyssarit = query(q)

In [5]:
df_columns = ['kysely', 'vuosineljannes', 'luokittelukoodi', 'kysymys_pitka', 'kyselyita', 'vastauksia', 'keskiarvo'
]
luokittelusuodatus = ["AIKTEHOHO", "EALAPSAIK", "ENSIHOITO"]
df_kooste_hopp = df_kooste_hopp[df_columns]
df_kooste_hopp = df_kooste_hopp[df_kooste_hopp["luokittelukoodi"].isin(luokittelusuodatus)]


In [6]:
# manuaalista putsailua silvertaulujen epätäydellisyyden takia
import re
def standardize(col):
    col = re.sub(r'\b0(\d+)\b', r'\1', col.strip())
    return col.strip().replace(' ', '_').replace('ä', 'a').replace('\n', '').replace('(', '').replace(')', '').replace('/', '_').replace(',', '').replace('.', '').lower().replace('ö', 'o')

df_kooste_hopp = df_kooste_hopp[~df_kooste_hopp['kysymys_pitka'].str.contains(r"(lapseni|Lapseni|lapselleni)", na=False)]


df_kooste_hopp = df_kooste_hopp[df_kooste_hopp['kysymys_pitka'] !=
    '11 Hoitajat auttoivat minua, jos minuun sattui']

df_kooste_hopp = df_kooste_hopp[df_kooste_hopp['kysymys_pitka'] !=
    '12 Hoitajilla oli aikaa kuunnella minua']

df_kooste_hopp = df_kooste_hopp[df_kooste_hopp['kysymys_pitka'] !=
    '13 Hoitajat kuuntelivat meitä huolellisesti']

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '01 Hoitajat ja minä suunnittelimme hoitoani yhdessä',
    '01 Hoitajat ottivat mielipiteeni huomioon, kun hoitoani suunniteltiin tai toteutettiin'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '03 Hoitajat hoitivat asian, josta sanoin, että se pitää korjata',
    '03 Hoitoni oli hyvin suunniteltu ja toteutettu hoitajien ja lääkäreiden toimesta'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '05 Hoitajat puhuivat kanssani niin, ettei kukaan minulle tuntematon kuullut asioitani',
    '08 Hoitajat puhuivat arkaluontoisista asioista siten, etteivät ulkopuoliset kuulleet niitä'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '04 Hoitajat kertoivat minulle, miksi lääkettä annetaan',
    '06 Hoitajat kertoivat minulle uuden lääkkeen antamisen yhteydessä, miksi lääkettä annetaan'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '08 Hoitajat olivat kohteliaita',
    '10 Hoitajat kohtelivat minua hyvin'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '06 Hoitajat olivat minulle ystävällisiä',
    '10 hoitajat kohtelivat minua hyvin'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '07 Hoitajat huolehtivat, ettei hoito tai tutkimukset aiheuttaneet minulle noloja tilanteita',
    '11 hoitajat huolehtivat etteivat hoito ja tai tutkimukset aiheuttaneet minulle noloja tai kiusallisia tilanteita'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '07 Hoitajat puhuivat arkaluontoisista asioista siten, etteivät ulkopuoliset kuulleet niitä',
    '11 hoitajat huolehtivat etteivat hoito ja tai tutkimukset aiheuttaneet minulle noloja tai kiusallisia tilanteita'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '08 Hoitajat auttoivat minua aina, kun tarvitsin apua',
    '13 sain tarvitsemani avun hoitajilta riittavan nopeasti'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '09 Hoitajat kertoivat minulle, miksi ja miten minua hoidetaan',
    '15 hoitajat selittivat ymmarrettavasti hoitooni ja tutkimuksiini liittyvat asiat'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '10 Hoitajat kertoivat minulle asioista niin, että minun oli helppo ymmärtää ne',
    '18 hoitajat varmistivat etta ymmarsin saamani tiedon'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '11 Hoitajat auttoivat minua, jos minuun sattui',
    '18 hoitajat varmistivat etta ymmarsin saamani tiedon'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '11 Hoitajat huolehtivat, etteivät hoito ja/tai tutkimukset aiheuttaneet minulle noloja tai kiusallisia tilanteita',
    '11 hoitajat huolehtivat etteivat hoito ja tai tutkimukset aiheuttaneet minulle noloja tai kiusallisia tilanteita'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '14 Hoitajat olivat aidosti läsnä',
    '22 Hoitajat olivat aidosti läsnä'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '15 Hoitajat selittivät ymmärrettävästi hoitooni ja tutkimuksiini liittyvät asiat',
    '15 hoitajat selittivat ymmarrettavasti hoitooni ja tutkimuksiini liittyvat asiat'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '18 Hoitajat varmistivat, että ymmärsin saamani tiedon',
    '18 hoitajat varmistivat etta ymmarsin saamani tiedon'
)
df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '10 Hoitajat kohtelivat minua hyvin',
    '10 hoitajat kohtelivat minua hyvin'
)
df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '18 Hoitajat varmistivat, että ymmärsin saamani tiedon',
    '18_hoitajat_varmistivat_etta_ymmarsin_saamani_tiedon'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '13 sain tarvitsemani avun hoitajilta riittavan nopeasti',
    '13 Sain tarvitsemani avun hoitajilta riittävän nopeasti'
)
df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '02 Hoitajat pyysivät minulta anteeksi, jos hoidossa tapahtui virhe',
    '04 Hoitajat pyysivät minulta anteeksi, jos hoidossani tapahtui virhe'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '02 Hoitajat pyysivät minulta anteeksi, jos hoidossa tapahtui virhe',
    '04 Hoitajat pyysivät minulta anteeksi, jos hoidossani tapahtui virhe'
)

df_kooste_hopp['kysymys_pitka'] = df_kooste_hopp['kysymys_pitka'].replace(
    '03 Hoitoni oli hyvin suunniteltu ja toteutettu hoitajien ja lääkäreiden toimesta',
    '03 Hoitoni oli hyvin suunniteltu ja toteutettu hoitajien sekä lääkäreiden toimesta'
)
df_kooste_hopp['kysymys_pitka'].unique()


/tmp/ipykernel_31228/1060216867.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_kooste_hopp = df_kooste_hopp[~df_kooste_hopp['kysymys_pitka'].str.contains(r"(lapseni|Lapseni|lapselleni)", na=False)]


array(['01 Hoitajat ottivat mielipiteeni huomioon, kun hoitoani suunniteltiin tai toteutettiin',
       '02 Hoitajat ja lääkärit toimivat hyvin yhdessä hoitooni liittyvissä asioissa',
       '03 Hoitoni oli hyvin suunniteltu ja toteutettu hoitajien sekä lääkäreiden toimesta',
       '04 Hoitajat pyysivät minulta anteeksi, jos hoidossani tapahtui virhe',
       '05 Hoitajat puuttuivat epäkohtaan, josta mainitsin heille',
       '06 Hoitajat kertoivat minulle uuden lääkkeen antamisen yhteydessä, miksi lääkettä annetaan',
       '07 Hoitajat kertoivat minulle saamieni lääkkeiden mahdollisista sivuvaikutuksista',
       '08 Hoitajat puhuivat arkaluontoisista asioista siten, etteivät ulkopuoliset kuulleet niitä',
       '09 Hoitajat huolehtivat, että liikkuminen oli turvallista hoidon aikana',
       '10 hoitajat kohtelivat minua hyvin',
       '11 hoitajat huolehtivat etteivat hoito ja tai tutkimukset aiheuttaneet minulle noloja tai kiusallisia tilanteita',
       '12 Hoitajat huolehtivat 

## Seuraavaksi HOPP kyssärit

In [ ]:
df_columns = ['quarter', 'unit_code', '1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin',
       '2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa',
       '3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta',
       '4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe',
       '5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille',
       '6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan',
       '7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista',
       '8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita',
       '9_hoitajat_huolehtivat_etta_liikkuminen_oli_turvallista_hoidon_aikana',
       '10_hoitajat_kohtelivat_minua_hyvin',
       '11_hoitajat_huolehtivat_etteivat_hoito_ja_tai_tutkimukset_aiheuttaneet_minulle_noloja_tai_kiusallisia_tilanteita',
       '12_hoitajat_huolehtivat_yksityisyyteni_toteutumisesta',
       '13_sain_tarvitsemani_avun_hoitajilta_riittavan_nopeasti',
       '14_sain_hoitajilta_apua_riittavan_nopeasti_halutessani_wc:hen_tai_alusastian',
       '15_hoitajat_selittivat_ymmarrettavasti_hoitooni_ja_tutkimuksiini_liittyvat_asiat',
       '16_hoitajat_ohjasivat_ymmarrettavasti_jatkohoitooni_liittyvat_asiat',
       '17_hoitajat_ohjasivat_ymmarrettavasti_kotona_vointini_tarkkailuun_liittyvat_asiat',
       '18_hoitajat_varmistivat_etta_ymmarsin_saamani_tiedon',
       '19_hoitaja_huolehti_etta_sain_lievitysta_kipuihin_kun_siihen_oli_tarvetta',
       '20_hoitajat_arvioivat_kipujani_riittavan_usein',
       '21_hoitajat_kuuntelivat_minua_huolellisesti',
       '22_hoitajat_olivat_aidosti_lasna']

df_kyssarit = df_kyssarit[df_columns]
df_kyssarit = df_kyssarit[df_kyssarit["unit_code"].isin(luokittelusuodatus)]

In [8]:
# muuta e arvot nan arvoiksi
df_kyssarit.replace("E", np.nan, inplace=True)
# etsi ja poista nan rivit
df_kyssarit.dropna(how="all", inplace=True)


/tmp/ipykernel_31228/1967167341.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_kyssarit.replace("E", np.nan, inplace=True)


In [9]:
df_kyssarit

,quarter,unit_code,1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin,2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa,3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta,4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe,5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille,6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan,7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista,8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita,...,13_sain_tarvitsemani_avun_hoitajilta_riittavan_nopeasti,14_sain_hoitajilta_apua_riittavan_nopeasti_halutessani_wc:hen_tai_alusastian,15_hoitajat_selittivat_ymmarrettavasti_hoitooni_ja_tutkimuksiini_liittyvat_asiat,16_hoitajat_ohjasivat_ymmarrettavasti_jatkohoitooni_liittyvat_asiat,17_hoitajat_ohjasivat_ymmarrettavasti_kotona_vointini_tarkkailuun_liittyvat_asiat,18_hoitajat_varmistivat_etta_ymmarsin_saamani_tiedon,19_hoitaja_huolehti_etta_sain_lievitysta_kipuihin_kun_siihen_oli_tarvetta,20_hoitajat_arvioivat_kipujani_riittavan_usein,21_hoitajat_kuuntelivat_minua_huolellisesti,22_hoitajat_olivat_aidosti_lasna
33,2_2023,EALAPSAIK,4.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
34,2_2023,EALAPSAIK,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2_2023,ENSIHOITO,5.0,5.0,5.0,NaN,NaN,5.0,NaN,5.0,...,5.0,NaN,5.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0
36,2_2023,ENSIHOITO,5.0,5.0,5.0,5.0,5.0,5.0,2.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
37,2_2023,ENSIHOITO,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,4_2022,ENSIHOITO,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,5.0,NaN,5.0,5.0,5.0,NaN,5.0,5.0
2160,4_2022,ENSIHOITO,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,5.0,NaN,NaN,3.0,5.0,4.0,4.0,4.0
2161,4_2022,ENSIHOITO,4.0,5.0,4.0,NaN,4.0,5.0,5.0,4.0,...,4.0,NaN,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2162,4_2022,ENSIHOITO,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,5.0,5.0,4.0,4.0,NaN,NaN,5.0,5.0


In [10]:
## kysymysten keskiarvo ryhmiteltynä kvartaalin mukaan ##
df_kyssarit_1 = df_kyssarit.groupby("quarter").mean(numeric_only=True).reset_index()
df_kyssarit_1["quarter"] = df_kyssarit_1["quarter"].apply(lambda x: f"{int(x.split('_')[0]):02d}_{x.split('_')[1]}")
df_kyssarit_1

,quarter,1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin,2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa,3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta,4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe,5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille,6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan,7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista,8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita,9_hoitajat_huolehtivat_etta_liikkuminen_oli_turvallista_hoidon_aikana,...,13_sain_tarvitsemani_avun_hoitajilta_riittavan_nopeasti,14_sain_hoitajilta_apua_riittavan_nopeasti_halutessani_wc:hen_tai_alusastian,15_hoitajat_selittivat_ymmarrettavasti_hoitooni_ja_tutkimuksiini_liittyvat_asiat,16_hoitajat_ohjasivat_ymmarrettavasti_jatkohoitooni_liittyvat_asiat,17_hoitajat_ohjasivat_ymmarrettavasti_kotona_vointini_tarkkailuun_liittyvat_asiat,18_hoitajat_varmistivat_etta_ymmarsin_saamani_tiedon,19_hoitaja_huolehti_etta_sain_lievitysta_kipuihin_kun_siihen_oli_tarvetta,20_hoitajat_arvioivat_kipujani_riittavan_usein,21_hoitajat_kuuntelivat_minua_huolellisesti,22_hoitajat_olivat_aidosti_lasna
0,01_2022,4.500000,4.727273,4.454545,4.000000,5.000000,4.700000,3.900000,4.888889,4.800000,...,4.500000,5.000000,4.818182,4.777778,4.714286,4.400000,4.750000,4.700000,5.000000,5.000000
1,02_2023,4.615385,4.666667,4.545455,3.333333,3.500000,4.000000,3.666667,4.166667,4.500000,...,4.916667,4.333333,4.916667,4.727273,4.375000,4.714286,4.555556,4.500000,4.909091,4.846154
2,03_2021,4.800000,4.647059,4.611111,4.571429,4.500000,NaN,NaN,4.800000,4.842105,...,4.666667,4.666667,4.800000,4.722222,4.666667,4.789474,4.888889,4.923077,4.777778,4.833333
3,03_2022,4.706667,4.750000,4.652174,4.533333,4.400000,4.465116,4.195122,4.515152,4.763636,...,4.661538,4.468750,4.761905,4.698413,4.634615,4.750000,4.645833,4.520833,4.835821,4.865672
4,03_2023,4.200000,4.000000,3.952381,4.444444,4.076923,4.692308,4.428571,4.833333,4.428571,...,4.000000,4.833333,4.705882,4.500000,4.307692,4.466667,4.727273,4.727273,4.529412,4.529412
5,04_2022,4.444444,4.538462,4.375000,3.333333,3.666667,4.818182,4.750000,4.666667,4.461538,...,4.411765,4.833333,4.529412,4.416667,4.666667,4.375000,4.333333,4.090909,4.400000,4.400000


In [11]:
df_kooste_hopp["quarter"] = df_kooste_hopp["vuosineljannes"].astype(str).apply(lambda x: f"{x[4:]}_{x[:4]}")

## pivotti

df_kooste_hopp = df_kooste_hopp.pivot_table(
    index="quarter",  # Group by 'vuosineljannes'
    columns="kysymys_pitka",  # Use 'kysymys_pitka' as columns
    values=["keskiarvo"],  # Aggregate these columns
    aggfunc="mean"  # Calculate the mean
)
# Flatten the column MultiIndex for better readability
df_kooste_hopp.columns = [col[1] for col in df_kooste_hopp.columns.values]
# Reset index for a clean table
df_kooste_hopp = df_kooste_hopp.reset_index()

df_kooste_hopp.columns = [standardize(col) for col in df_kooste_hopp.columns]

df_kooste_hopp


,quarter,1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin,2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa,3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta,4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe,5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille,6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan,7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista,8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita,9_hoitajat_huolehtivat_etta_liikkuminen_oli_turvallista_hoidon_aikana,...,13_sain_tarvitsemani_avun_hoitajilta_riittavan_nopeasti,14_sain_hoitajilta_apua_riittavan_nopeasti_halutessani_wc:hen_tai_alusastian,15_hoitajat_selittivat_ymmarrettavasti_hoitooni_ja_tutkimuksiini_liittyvat_asiat,16_hoitajat_ohjasivat_ymmarrettavasti_jatkohoitooni_liittyvat_asiat,17_hoitajat_ohjasivat_ymmarrettavasti_kotona_vointini_tarkkailuun_liittyvat_asiat,18_hoitajat_varmistivat_etta_ymmarsin_saamani_tiedon,19_hoitaja_huolehti_etta_sain_lievitysta_kipuihin_kun_siihen_oli_tarvetta,20_hoitajat_arvioivat_kipujani_riittavan_usein,21_hoitajat_kuuntelivat_minua_huolellisesti,22_hoitajat_olivat_aidosti_lasna
0,01_2022,4.150000,4.073333,4.046667,3.680000,3.496667,4.336667,3.800000,4.030000,4.426667,...,4.106667,3.995000,4.280000,4.366667,4.166667,4.156667,4.233333,4.240000,4.406667,4.310000
1,01_2023,4.566667,4.640000,4.585000,4.440000,3.940000,4.560000,4.330000,4.840000,4.750000,...,4.716667,4.920000,4.786667,4.885000,4.590000,4.735000,4.785000,4.650000,4.750000,4.623333
2,02_2022,4.336667,4.403333,4.323333,4.130000,3.845000,4.043333,3.663333,4.250000,4.503333,...,4.270000,4.140000,4.436667,4.186667,4.146667,4.403333,4.453333,4.186667,4.376667,4.237500
3,02_2023,4.340000,4.375000,4.315000,4.015000,3.653333,4.380000,3.995000,4.355000,4.450000,...,4.620000,4.130000,4.485000,4.390000,4.250000,4.390000,4.480000,4.370000,4.480000,4.333333
4,03_2021,4.476667,4.253333,4.270000,4.040000,3.353333,4.503333,3.660000,4.423333,4.606667,...,4.216667,4.575000,4.466667,4.296667,4.440000,4.490000,4.453333,4.393333,4.406667,4.095000
5,03_2022,4.560000,4.483333,4.530000,4.076667,4.025000,4.416667,4.183333,4.343333,4.656667,...,4.340000,4.296667,4.483333,4.426667,4.210000,4.493333,4.436667,4.193333,4.496667,4.492500
6,04_2021,4.230000,4.460000,4.330000,3.965000,3.880000,4.300000,4.010000,4.175000,4.460000,...,4.295000,4.440000,4.415000,4.385000,4.240000,4.426667,4.295000,4.270000,4.540000,4.372500
7,04_2022,4.392500,4.246667,4.320000,3.500000,3.303333,4.560000,4.083333,4.457500,4.523333,...,4.510000,4.430000,4.525000,4.356667,4.130000,4.547500,4.320000,4.216667,4.380000,4.315000


In [12]:
df_kooste_hopp.columns

Index(['quarter',
       '1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin',
       '2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa',
       '3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta',
       '4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe',
       '5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille',
       '6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan',
       '7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista',
       '8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita',
       '9_hoitajat_huolehtivat_etta_liikkuminen_oli_turvallista_hoidon_aikana',
       '10_hoitajat_kohtelivat_minua_hyvin',
       '11_hoitajat_huolehtivat_etteivat_hoito_ja_tai_tutkimukset_aiheuttaneet_minulle_noloja_tai_kiusallisia_tilanteita',
       '12_hoitajat_huolehtiv

In [13]:
 ## yhdistetään kaksi dataframea yhdeksi ##
## kooste ja hopp kainuun kyselyt yhdistetään samoihin kysymyksiin ja ne erotellaan "datajoukko" perusteella ##

df_kooste_hopp.insert(1, "datajoukko", None)

df_kooste_hopp["datajoukko"] = "kooste"
df_kyssarit_1["datajoukko"] = "kainuu"

In [14]:

df_combined = pd.concat([df_kooste_hopp, df_kyssarit_1], ignore_index=True)

# Display the combined DataFrame
list(df_combined.columns)

['quarter',
 'datajoukko',
 '1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin',
 '2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa',
 '3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta',
 '4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe',
 '5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille',
 '6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan',
 '7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista',
 '8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita',
 '9_hoitajat_huolehtivat_etta_liikkuminen_oli_turvallista_hoidon_aikana',
 '10_hoitajat_kohtelivat_minua_hyvin',
 '11_hoitajat_huolehtivat_etteivat_hoito_ja_tai_tutkimukset_aiheuttaneet_minulle_noloja_tai_kiusallisia_tilanteita',
 '12_hoitajat_huolehtivat_yksityisyyteni_toteutumisesta',
 '13_sain_tarvitsemani_avun_

In [15]:
df_combined

,quarter,datajoukko,1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin,2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa,3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta,4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe,5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille,6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan,7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista,8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita,...,13_sain_tarvitsemani_avun_hoitajilta_riittavan_nopeasti,14_sain_hoitajilta_apua_riittavan_nopeasti_halutessani_wc:hen_tai_alusastian,15_hoitajat_selittivat_ymmarrettavasti_hoitooni_ja_tutkimuksiini_liittyvat_asiat,16_hoitajat_ohjasivat_ymmarrettavasti_jatkohoitooni_liittyvat_asiat,17_hoitajat_ohjasivat_ymmarrettavasti_kotona_vointini_tarkkailuun_liittyvat_asiat,18_hoitajat_varmistivat_etta_ymmarsin_saamani_tiedon,19_hoitaja_huolehti_etta_sain_lievitysta_kipuihin_kun_siihen_oli_tarvetta,20_hoitajat_arvioivat_kipujani_riittavan_usein,21_hoitajat_kuuntelivat_minua_huolellisesti,22_hoitajat_olivat_aidosti_lasna
0,01_2022,kooste,4.150000,4.073333,4.046667,3.680000,3.496667,4.336667,3.800000,4.030000,...,4.106667,3.995000,4.280000,4.366667,4.166667,4.156667,4.233333,4.240000,4.406667,4.310000
1,01_2023,kooste,4.566667,4.640000,4.585000,4.440000,3.940000,4.560000,4.330000,4.840000,...,4.716667,4.920000,4.786667,4.885000,4.590000,4.735000,4.785000,4.650000,4.750000,4.623333
2,02_2022,kooste,4.336667,4.403333,4.323333,4.130000,3.845000,4.043333,3.663333,4.250000,...,4.270000,4.140000,4.436667,4.186667,4.146667,4.403333,4.453333,4.186667,4.376667,4.237500
3,02_2023,kooste,4.340000,4.375000,4.315000,4.015000,3.653333,4.380000,3.995000,4.355000,...,4.620000,4.130000,4.485000,4.390000,4.250000,4.390000,4.480000,4.370000,4.480000,4.333333
4,03_2021,kooste,4.476667,4.253333,4.270000,4.040000,3.353333,4.503333,3.660000,4.423333,...,4.216667,4.575000,4.466667,4.296667,4.440000,4.490000,4.453333,4.393333,4.406667,4.095000
5,03_2022,kooste,4.560000,4.483333,4.530000,4.076667,4.025000,4.416667,4.183333,4.343333,...,4.340000,4.296667,4.483333,4.426667,4.210000,4.493333,4.436667,4.193333,4.496667,4.492500
6,04_2021,kooste,4.230000,4.460000,4.330000,3.965000,3.880000,4.300000,4.010000,4.175000,...,4.295000,4.440000,4.415000,4.385000,4.240000,4.426667,4.295000,4.270000,4.540000,4.372500
7,04_2022,kooste,4.392500,4.246667,4.320000,3.500000,3.303333,4.560000,4.083333,4.457500,...,4.510000,4.430000,4.525000,4.356667,4.130000,4.547500,4.320000,4.216667,4.380000,4.315000
8,01_2022,kainuu,4.500000,4.727273,4.454545,4.000000,5.000000,4.700000,3.900000,4.888889,...,4.500000,5.000000,4.818182,4.777778,4.714286,4.400000,4.750000,4.700000,5.000000,5.000000
9,02_2023,kainuu,4.615385,4.666667,4.545455,3.333333,3.500000,4.000000,3.666667,4.166667,...,4.916667,4.333333,4.916667,4.727273,4.375000,4.714286,4.555556,4.500000,4.909091,4.846154
